# Chatbot Educativo con Naive Bayes y Similitud Semántica
Este notebook desarrolla un chatbot que clasifica las preguntas de los estudiantes y selecciona la mejor respuesta posible con base en análisis semántico.

## ¿Qué es Naive Bayes?
Naive Bayes es un clasificador probabilístico basado en el teorema de Bayes. Asume que las características (palabras, en este caso) son independientes entre sí. Es eficaz para tareas de clasificación de texto, como spam, sentimientos o, en este caso, la clasificación de intenciones.

## ¿Qué es TF-IDF?
**TF-IDF** (Term Frequency - Inverse Document Frequency) es una técnica que transforma texto en valores numéricos que reflejan la importancia de las palabras en los documentos. Se utiliza para vectorizar texto antes de aplicar modelos de aprendizaje automático.

## ¿Qué es la similitud semántica?
La **similitud semántica** mide cuán similares son dos textos en términos de significado. En este caso usamos vectores de SpaCy para representar frases, y comparamos la consulta del usuario con las posibles respuestas para elegir la más adecuada.

## 📐 Fundamentos Matemáticos

### 🔢 Teorema de Bayes aplicado a texto
El clasificador Naive Bayes predice la clase \( C_k \) dada una observación \( x \) usando la siguiente fórmula:

\[ P(C_k \mid x) = \frac{P(x \mid C_k) P(C_k)}{P(x)} \]

En clasificación de texto, \( x \) son las palabras del documento. Bajo la suposición de independencia ingenua entre palabras, se convierte en:

\[ P(C_k \mid x_1, ..., x_n) \propto P(C_k) \prod_{i=1}^n P(x_i \mid C_k) \]

### 🧮 Fórmula de TF-IDF
**TF (Frecuencia de Término):** cuántas veces aparece un término en un documento.

\[ TF(t, d) = \frac{f_{t,d}}{\sum_k f_{k,d}} \]

**IDF (Frecuencia Inversa de Documentos):** mide cuán común o rara es una palabra en todos los documentos.

\[ IDF(t, D) = \log \left( \frac{N}{1 + n_t} \right) \]

Donde:
- \( f_{t,d} \) es la frecuencia del término \( t \) en el documento \( d \)
- \( N \) es el número total de documentos
- \( n_t \) es el número de documentos que contienen el término \( t \)

**TF-IDF final:**
\[ TFIDF(t, d, D) = TF(t, d) \cdot IDF(t, D) \]

### 🔗 Similitud de coseno
Se usa para medir qué tan similares son dos vectores (frase del usuario y respuesta).
Se calcula así:

\[ \text{coseno}(A, B) = \frac{A \cdot B}{\|A\| \|B\|} = \frac{\sum_{i=1}^n A_i B_i}{\sqrt{\sum_{i=1}^n A_i^2} \sqrt{\sum_{i=1}^n B_i^2}} \]

El valor resultante está entre 0 (completamente diferente) y 1 (idéntico).

# Chatbot Educativo con Naive Bayes y TF-IDF

### Entrenamiento del clasificador con TF-IDF y Naive Bayes
Creamos un pipeline que convierte las frases limpias en vectores usando TF-IDF, y luego las clasifica con un modelo de Naive Bayes entrenado sobre las etiquetas del dataset.

**Chatbot Educativo con Naive Bayes y TF-IDF**
Este cuaderno entrena un modelo Naive Bayes para responder preguntas frecuentes sobre temas académicos como matrículas, horarios, calificaciones, etc.


In [66]:
#importar libreria
import pandas as pd
import spacy
import unicodedata
import re
from sklearn.metrics.pairwise import cosine_similarity
import random
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

nlp = spacy.load("es_core_news_md")


In [67]:
## 📥 Cargar el Dataset
df = pd.read_csv(r"C:\Users\darly\OneDrive\Escritorio\materialClaseIA\claseFIN8\dataEducacion_final_393_agregado_2.csv")

df.head()

,consulta,categoria,respuesta,respuestas_compuestas,consulta_
0,¿Cómo me inscribo a clases el próximo semestre?,matriculas,"Para inscribirte al próximo semestre, debes in...",¿Cómo me inscribo a clases el próximo semestre...,¿Cómo me inscribo a clases el próximo semestre...
1,¿Cuál es la fecha límite para inscribirse?,matriculas,La fecha límite para completar tu inscripción ...,¿Cuál es la fecha límite para inscribirse? La ...,¿Cuál es la fecha límite para inscribirse? mat...
2,¿Qué documentos necesito para la matrícula?,matriculas,Para realizar tu matrícula necesitas: identifi...,¿Qué documentos necesito para la matrícula? Pa...,¿Qué documentos necesito para la matrícula? ma...
3,¿Cuánto cuesta la inscripción?,matriculas,El costo de inscripción para el semestre actua...,¿Cuánto cuesta la inscripción? El costo de ins...,¿Cuánto cuesta la inscripción? matriculas
4,¿Dónde puedo pagar mi matrícula?,matriculas,Puedes pagar tu matrícula en cualquier sucursa...,¿Dónde puedo pagar mi matrícula? Puedes pagar ...,¿Dónde puedo pagar mi matrícula? matriculas


In [68]:
#limpieza de los datos #Incluye:
#- Minúsculas
#- Eliminación de tildes
#- Lematización
#- Conserva palabras importantes como "dónde", "cuándo", "cómo", "qué", etc.


# Palabras que queremos conservar (normalizadas)
custom_stopwords_to_keep = {"no", "si", "donde", "cuando", "como", "que", "cual", "cuanto"}

def normalize_text(text):
    # Paso 1: Convertir a minúsculas
    text = text.lower()

    # Paso 2: Eliminar tildes y ñ
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')  # elimina tildes y ñ

    # Paso 3: Eliminar signos de puntuación
    text = re.sub(r"[^\w\s]", "", text)

    # Paso 4: Eliminar espacios extra
    text = re.sub(r"\s+", " ", text).strip()

    # Paso 5: Procesar con spaCy
    doc = nlp(text)

    # Paso 6: Lematizar, eliminar stopwords y quitar tildes también a los lemas
    tokens = [
        ''.join(
            c for c in unicodedata.normalize('NFD', token.lemma_)
            if unicodedata.category(c) != 'Mn'
        )
        for token in doc
        if not (token.is_stop and token.lemma_.lower() not in custom_stopwords_to_keep)
    ]

    return " ".join(tokens)



In [69]:
# Aplicar limpieza
df["consulta_limpia"] = df["consulta"].apply(normalize_text)

In [70]:
df

,consulta,categoria,respuesta,respuestas_compuestas,consulta_,consulta_limpia
0,¿Cómo me inscribo a clases el próximo semestre?,matriculas,"Para inscribirte al próximo semestre, debes in...",¿Cómo me inscribo a clases el próximo semestre...,¿Cómo me inscribo a clases el próximo semestre...,como inscribo clase semestre
1,¿Cuál es la fecha límite para inscribirse?,matriculas,La fecha límite para completar tu inscripción ...,¿Cuál es la fecha límite para inscribirse? La ...,¿Cuál es la fecha límite para inscribirse? mat...,cual fecha limite inscribir el
2,¿Qué documentos necesito para la matrícula?,matriculas,Para realizar tu matrícula necesitas: identifi...,¿Qué documentos necesito para la matrícula? Pa...,¿Qué documentos necesito para la matrícula? ma...,que documento necesitar matricula
3,¿Cuánto cuesta la inscripción?,matriculas,El costo de inscripción para el semestre actua...,¿Cuánto cuesta la inscripción? El costo de ins...,¿Cuánto cuesta la inscripción? matriculas,cuanto costar inscripcion
4,¿Dónde puedo pagar mi matrícula?,matriculas,Puedes pagar tu matrícula en cualquier sucursa...,¿Dónde puedo pagar mi matrícula? Puedes pagar ...,¿Dónde puedo pagar mi matrícula? matriculas,donde pagar matricula
...,...,...,...,...,...,...
388,¿Se puede anular una calificación registrada p...,calificaciones,"Sí, si el docente lo justifica y se presenta e...",¿Se puede anular una calificación registrada p...,¿Se puede anular una calificación registrada p...,anular calificacion registrado error
389,¿Puedo ver el detalle de cómo se calculó mi no...,calificaciones,"Sí, puedes solicitar el desglose de calificaci...",¿Puedo ver el detalle de cómo se calculó mi no...,¿Puedo ver el detalle de cómo se calculó mi no...,detalle como calcular nota
390,¿Cuáles son las principales instalaciones de l...,instalaciones,Las principales instalaciones de la universida...,¿Cuáles son las principales instalaciones de l...,¿Cuáles son las principales instalaciones de l...,cual principal instalacion universidad
391,¿Cuáles son los principales requisitos que exi...,requisitos,Los principales requisitos que exige la univer...,¿Cuáles son los principales requisitos que exi...,¿Cuáles son los principales requisitos que exi...,cual principal requisito que exigir universida...


In [71]:
#Vamos a crear un modelo de clasificación de intención por categoría usando n-gramas de 1 a 3.

X = df["consulta_limpia"]
y = df["categoria"]

In [72]:
#partir set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
#crear estructura de entranamiento y medir la clasificacion

model = make_pipeline(
    TfidfVectorizer(
        ngram_range=(1, 2),       # Usa un contexto de hasta 3 palabras
        min_df=1                # Aparece en al menos 2 consulta
                   ),
    MultinomialNB()
)


In [74]:
#entrenar modelo
model.fit(X_train, y_train)
print("Precisión del modelo:", model.score(X_test, y_test))
preds = model.predict(X_test)
print(classification_report(y_test, preds))


Precisión del modelo: 0.8481012658227848
                precision    recall  f1-score   support

calificaciones       0.93      0.93      0.93        14
      horarios       0.76      0.93      0.84        14
 instalaciones       1.00      0.92      0.96        12
    matriculas       0.82      0.82      0.82        11
    requisitos       0.79      0.79      0.79        14
      tramites       0.83      0.71      0.77        14

      accuracy                           0.85        79
     macro avg       0.86      0.85      0.85        79
  weighted avg       0.85      0.85      0.85        79



In [75]:
## 💬 Motor de Chatbot
#Definimos saludos y despedidas y usamos clasificación + similitud para elegir la mejor respuesta en cada categoría.

saludos = ["hola", "buenos dias", "buenas tardes", "buenas noches", "hola que tal", "como estas", "ey"]
respuestas_saludo = ["¡Hola! ¿En qué puedo ayudarte?", "¡Buenos días! ¿Qué necesitas?", "Hola, dime tu duda."]

despedidas = ["gracias", "hasta luego", "adios", "nos vemos", "bye", "chao"]
respuestas_despedida = ["¡Hasta pronto!", "Gracias por tu consulta. ¡Éxitos!", "Nos vemos."]

def responder(pregunta):
    pregunta_limpia = normalize_text(pregunta)
    print("🔍 Consulta limpia →", pregunta_limpia)
    
    if any(saludo in pregunta.lower() for saludo in saludos):
        return random.choice(respuestas_saludo)
    
    if any(despedida in pregunta.lower() for despedida in despedidas):
        return random.choice(respuestas_despedida)
    
    # 1. Clasificar la intención (etiqueta)
    etiqueta = model.predict([pregunta_limpia])[0]
    print("📌 Categoría detectada →", etiqueta)
    
    # 2. Filtrar respuestas de esa categoría
    respuestas_categoria = df[df["categoria"] == etiqueta].copy()
    

    # 3. Normalizar las respuestas para compararlas
    respuestas_categoria["respuesta_limpia"] = respuestas_categoria["respuestas_compuestas"].apply(normalize_text)
    #print(respuestas_categoria)

    # 4. Vectorizar respuestas limpias
    vectorizer = model.named_steps["tfidfvectorizer"]
    respuestas_categoria["respuesta_vec"] = respuestas_categoria["respuesta_limpia"].apply(lambda x: vectorizer.transform([x]))
    
    pregunta_vec = vectorizer.transform([pregunta_limpia])
    print("_________________________________________________________________________________________")

    print(respuestas_categoria)
    print("_________________________________________________________________________________________FIN")
    
    # 5. Calcular similitud contra respuestas
    respuestas_categoria["similitud"] = respuestas_categoria["respuesta_vec"].apply(lambda x: cosine_similarity(x, pregunta_vec)[0][0])

    # 6. Mostrar top 5 respuestas más similares
    print("\n🎯 Top 5 respuestas más similares:")
    top5 = respuestas_categoria[["respuestas_compuestas", "similitud"]].sort_values(by="similitud", ascending=False).head(5)
    print(top5.to_string(index=False))

    # 7. Devolver la mejor respuesta
    mejor_idx = respuestas_categoria["similitud"].idxmax()
    

    print("\n______________________________PREGUNTA__________________________")
    print(pregunta)
    print("\n______________________________RESPUESTA__________________________")

    return df.loc[mejor_idx, "respuesta"]


In [76]:
#prueba
print(responder("hola"))

🔍 Consulta limpia → hola
¡Hola! ¿En qué puedo ayudarte?


In [77]:
print(responder("¿Cómo me inscribo a clases?"))

🔍 Consulta limpia → como inscribo clase
📌 Categoría detectada → horarios
_________________________________________________________________________________________
                                              consulta categoria  \
15                   ¿A qué hora comienzan las clases?  horarios   
16                        ¿Dónde puedo ver mi horario?  horarios   
17                 ¿Cuándo termina el semestre actual?  horarios   
18   ¿Cuáles son los horarios de atención de secret...  horarios   
19                ¿Hay clases durante la semana santa?  horarios   
..                                                 ...       ...   
244           ¿Cómo se asignan los grupos por horario?  horarios   
245  ¿Puedo tomar una clase más tarde por razones d...  horarios   
246  ¿Dónde consulto el horario de eventos instituc...  horarios   
247           ¿Cómo saber si el horario es definitivo?  horarios   
248   ¿Qué pasa si no tengo clases en un día completo?  horarios   

                    

In [78]:
print(responder("cuales son las instalaciones de la universidad"))

🔍 Consulta limpia → cual instalacion universidad
📌 Categoría detectada → instalaciones
_________________________________________________________________________________________
                                              consulta      categoria  \
43                          ¿Dónde está la biblioteca?  instalaciones   
44     ¿A qué hora abre el laboratorio de informática?  instalaciones   
45                ¿Cómo reservo un salón para estudio?  instalaciones   
46                          ¿Dónde queda la cafetería?  instalaciones   
47              ¿Hay estacionamiento para estudiantes?  instalaciones   
..                                                 ...            ...   
336            ¿Dónde encuentro los planos del campus?  instalaciones   
337  ¿Qué servicios ofrece la oficina de bienestar ...  instalaciones   
338  ¿Puedo organizar una exposición en la zona cul...  instalaciones   
368  ¿Qué servicios tecnológicos ofrece la universi...  instalaciones   
390  ¿Cuáles son las

In [82]:
responder("como puedo registrarme a clases los fines de semana?")

🔍 Consulta limpia → como registrar yo clase fin semana
📌 Categoría detectada → horarios
_________________________________________________________________________________________
                                              consulta categoria  \
15                   ¿A qué hora comienzan las clases?  horarios   
16                        ¿Dónde puedo ver mi horario?  horarios   
17                 ¿Cuándo termina el semestre actual?  horarios   
18   ¿Cuáles son los horarios de atención de secret...  horarios   
19                ¿Hay clases durante la semana santa?  horarios   
..                                                 ...       ...   
244           ¿Cómo se asignan los grupos por horario?  horarios   
245  ¿Puedo tomar una clase más tarde por razones d...  horarios   
246  ¿Dónde consulto el horario de eventos instituc...  horarios   
247           ¿Cómo saber si el horario es definitivo?  horarios   
248   ¿Qué pasa si no tengo clases en un día completo?  horarios   

     

'Sí, hay materias disponibles los sábados. Verifica en la oferta académica al momento de inscribirte.'

In [84]:
responder("hay instalaciones de parqueo o estacionamiento en la universidad para estudiantes")

🔍 Consulta limpia → instalacion parqueo estacionamiento universidad estudiante
📌 Categoría detectada → instalaciones
_________________________________________________________________________________________
                                              consulta      categoria  \
43                          ¿Dónde está la biblioteca?  instalaciones   
44     ¿A qué hora abre el laboratorio de informática?  instalaciones   
45                ¿Cómo reservo un salón para estudio?  instalaciones   
46                          ¿Dónde queda la cafetería?  instalaciones   
47              ¿Hay estacionamiento para estudiantes?  instalaciones   
..                                                 ...            ...   
336            ¿Dónde encuentro los planos del campus?  instalaciones   
337  ¿Qué servicios ofrece la oficina de bienestar ...  instalaciones   
338  ¿Puedo organizar una exposición en la zona cul...  instalaciones   
368  ¿Qué servicios tecnológicos ofrece la universi...  instala

'Sí, contamos con estacionamiento para estudiantes en la parte posterior del campus. Para acceder necesitas un tarjetón vehicular que puedes solicitar en el departamento de seguridad presentando tarjeta de circulación, licencia de conducir vigente y credencial de estudiante.'

In [85]:
responder("hablame de los principales tramites de la universiadad")

🔍 Consulta limpia → hablamir principal tramit universiadad
📌 Categoría detectada → tramites
_________________________________________________________________________________________
                                              consulta categoria  \
29          ¿Cómo solicito una constancia de estudios?  tramites   
30   ¿En qué lugar puedo reclamar mi carné estudian...  tramites   
31                 ¿Cómo hago para cambiarme de grupo?  tramites   
32   ¿Cuál es el proceso para dar de baja una materia?  tramites   
33                ¿Cómo solicito un cambio de carrera?  tramites   
..                                                 ...       ...   
320          ¿Puedo tramitar documentos en vacaciones?  tramites   
321    ¿Qué es una certificación académica legalizada?  tramites   
322   ¿Cómo gestiono una baja definitiva del programa?  tramites   
323  ¿Dónde consulto el estado de mis trámites acad...  tramites   
392  ¿Cuáles son los principales trámites que hacen...  tramites   

 

'Entre los principales trámites que hacen los estudiantes en la universidad se encuentran la matrícula, solicitudes de certificados, inscripción a materias, cancelación de asignaturas y solicitud de homologaciones.'

In [86]:
responder("que servicios ofrece la universidad")

🔍 Consulta limpia → que servicio ofrecer universidad
📌 Categoría detectada → instalaciones
_________________________________________________________________________________________
                                              consulta      categoria  \
43                          ¿Dónde está la biblioteca?  instalaciones   
44     ¿A qué hora abre el laboratorio de informática?  instalaciones   
45                ¿Cómo reservo un salón para estudio?  instalaciones   
46                          ¿Dónde queda la cafetería?  instalaciones   
47              ¿Hay estacionamiento para estudiantes?  instalaciones   
..                                                 ...            ...   
336            ¿Dónde encuentro los planos del campus?  instalaciones   
337  ¿Qué servicios ofrece la oficina de bienestar ...  instalaciones   
338  ¿Puedo organizar una exposición en la zona cul...  instalaciones   
368  ¿Qué servicios tecnológicos ofrece la universi...  instalaciones   
390  ¿Cuáles son

'La enfermería ofrece primeros auxilios, control de signos vitales, atención básica y canalización a clínicas externas si es necesario.'

In [88]:
responder("tengo hambre, donde queda la cafetería")

🔍 Consulta limpia → hambre donde quedar cafeteria
📌 Categoría detectada → instalaciones
_________________________________________________________________________________________
                                              consulta      categoria  \
43                          ¿Dónde está la biblioteca?  instalaciones   
44     ¿A qué hora abre el laboratorio de informática?  instalaciones   
45                ¿Cómo reservo un salón para estudio?  instalaciones   
46                          ¿Dónde queda la cafetería?  instalaciones   
47              ¿Hay estacionamiento para estudiantes?  instalaciones   
..                                                 ...            ...   
336            ¿Dónde encuentro los planos del campus?  instalaciones   
337  ¿Qué servicios ofrece la oficina de bienestar ...  instalaciones   
338  ¿Puedo organizar una exposición en la zona cul...  instalaciones   
368  ¿Qué servicios tecnológicos ofrece la universi...  instalaciones   
390  ¿Cuáles son la

'La cafetería principal está ubicada entre los edificios B y C, con horario de 7:00 AM a 8:00 PM de lunes a viernes y de 8:00 AM a 2:00 PM los sábados. También hay cafeterías más pequeñas en los edificios D y F que operan en horario reducido.'